In [17]:
import numpy as np
import pandas as pd
import sklearn
import time
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from sklearn.model_selection import KFold
from scipy.sparse import find
from scipy.sparse import csr_matrix

In [3]:
data_path = '../datasets/'
data = np.load(data_path + 'ratings_train.npy')
data_test = np.nan_to_num(np.load(data_path + 'ratings_test.npy'))

In [4]:
data.shape

(610, 4980)

In [18]:
class Solve:

    def __init__(self, k, mu, alpha,beta, train_data, descent_method = 'SGD', n_steps = 100, seed = 10):
        self.k = k
        self.mu = mu
        self.alpha = alpha
        self.beta = beta
        self.data = np.copy(train_data)
        self.non_nan = np.argwhere(~np.isnan(train_data))
        self.descent = descent_method
        self.I = np.random.rand(len(self.data), self.k) # Generating random matrices, maybe a better initialization can be initialized
        self.U = np.random.rand(len(self.data[0]), self.k).T
        
        self.I_2 = np.random.rand(len(self.data), self.k) # Generating random matrices, maybe a better initialization can be initialized
        self.U_2 = np.random.rand(len(self.data[0]), self.k)
                
        self.n_steps = n_steps
        
    
    def compute_sgd(self):
        d_I, d_U = 0, 0
        for (i, j) in self.non_nan:
            eij = data[i][j] - np.dot(self.I[i,:],self.U[:,j])
            for k in range(self.k):
                d_I += self.I[i][k] + self.alpha * (2 * eij * self.U[k][j] - self.mu * self.I[i][k])
                d_U += self.U[k][j] + self.beta * (2 * eij * self.I[i][k] - self.mu * self.U[k][j])

        return d_I,d_U
    
    def train(self, output_loss=False):
        loss = []
        for _ in range(self.n_steps):
            if output_loss:
                e = 0
                for (i,j) in self.non_nan:
                    e = e + pow(self.data[i][j] - np.dot(self.I[i,:],self.U[:,j]), 2)
                    for k in range(self.k):
                        e = e + (self.mu/2) * (pow(self.I[i][k],2) + pow(self.U[k][j],2))

                loss.append(e)

            for (i, j) in self.non_nan:
                eij = self.data[i][j] - np.dot(self.I[i,:],self.U[:,j])
                for k in range(self.k):
                    self.I[i, k] = self.I[i, k] + self.alpha * (2 * eij * self.U[k, j] - self.mu * self.I[i, k])
                    self.U[k, j] = self.U[k, j] + self.beta * (2 * eij * self.I[i, k] - self.mu * self.U[k, j])

        return loss
    
    
    def matrix_completion_als(self, max_iter=125, tol=1e-6, lambda_reg=0.34):
        m, n = self.data.shape
        error = 1e10
        Omega = (self.data > 0).astype(int)
        print(f'Omega: {Omega}')
        for _ in range(max_iter):
            # Update U while fixing V
            for i in range(m):
                indices = np.where(Omega[i, :] == 1)[0]
                Vi = self.U_2[indices, :]
                Yi = self.data[i, indices]
                self.I_2[i, :] = np.linalg.solve(Vi.T @ Vi + lambda_reg * np.eye(self.k), Vi.T @ Yi)
                #print(f'Unique values: {np.unique(self.I_2)}')
        
            # Update V while fixing U
            for j in range(n):
                indices = np.where(Omega[:, j] == 1)[0]
                Uj = self.I_2[indices, :]
                Yj = self.data[indices, j]
                self.U_2[j, :] = np.linalg.solve(Uj.T @ Uj + lambda_reg * np.eye(self.k), Uj.T @ Yj)
                #print(f'Unique values: {np.unique(self.U_2)}')

            # Compute the matrix approximation and error
            Y_hat = self.I_2 @ self.U_2.T            
            diff = Omega * (self.data - Y_hat)
            new_error = np.linalg.norm(diff, 'fro')
            if abs(new_error - error) < tol:
                break
            error = new_error
    
    def train_k_folds(self, k_folds=5, max_iter=125, tol=1e-6, lambda_reg=0.34):
        kf = KFold(n_splits=k_folds)

        for train_idx, _ in kf.split(self.data):
            training_data = np.zeros_like(self.data)
            training_data[train_idx] = self.data
    

    def train_masked(self):
        for _ in range(self.n_steps):
            masked = np.ma.array(self.data, mask=np.isnan(self.data))
            masked_T = np.ma.transpose(masked)
            d_U = np.ma.add(np.ma.add(-2*np.ma.dot(masked_T,self.I), 2*self.U.T@self.I.T@self.I) , 2*self.mu*self.U.T)
            #d_U = np.ma.add(np.ma.add(-2*masked_T@self.I, 2*self.U@self.I.T@self.I),2*self.mu*self.U)
            d_I = np.ma.add(np.ma.add(-2*np.ma.dot(masked,self.U.T) ,2*self.I@self.U@self.U.T), 2*self.mu*self.I)
            self.I -= self.alpha*d_I
            self.U -= self.beta*d_U.T

    def rmse(self, test_matrix):
        masked = np.ma.array(test_matrix, mask=np.isnan(test_matrix))
        predictions = np.clip(np.around((self.I@self.U)*2, 0)/2, 1, 5)
        diff = np.ma.subtract(predictions, masked)
        squared = np.ma.power(diff, 2)
        return np.ma.sqrt(np.ma.mean(squared))
    
    def rmse_als(self, test_matrix):
        masked = np.ma.array(test_matrix, mask=np.isnan(test_matrix))
        predictions = np.clip(np.around((self.I_2@self.U_2.T)*2, 0)/2, 1, 5)
        diff = np.ma.subtract(predictions, masked)
        squared = np.ma.power(diff, 2)
        return np.ma.sqrt(np.ma.mean(squared))
    
    def rmse_als2(self, test_matrix):
        masked = np.ma.array(test_matrix, mask=np.isnan(test_matrix))
        predictions = self.I_2@self.U_2.T
        rows, cols = predictions.shape
        for i in range(rows):
            for j in range(cols):
                decimal_part = predictions[i, j] - int(predictions[i, j])
                if decimal_part < 0.45:
                    predictions[i, j] = np.floor(predictions[i, j])
                elif decimal_part > 0.55:
                    predictions[i, j] = np.ceil(predictions[i, j])
                else:
                    predictions[i, j] = int(predictions[i, j]) + 0.5
        diff = np.ma.subtract(predictions, masked)
        squared = np.ma.power(diff, 2)
        return np.ma.sqrt(np.ma.mean(squared))

    def predict(self):
        return np.clip(np.around((self.I@self.U)*2, 0)/2, 1, 5)
    
    def predict_als(self):
        return np.clip(np.around((self.I_2@self.U_2.T)*2, 0)/2, 1, 5)
    
    def predict_als2(self):
        pred = self.I_2@self.U_2.T
        rows, cols = pred.shape
        for i in range(rows):
            for j in range(cols):
                decimal_part = pred[i, j] - int(pred[i, j])
                if decimal_part < 0.45:
                    pred[i, j] = np.floor(pred[i, j])
                elif decimal_part > 0.55:
                    pred[i, j] = np.ceil(pred[i, j])
                else:
                    pred[i, j] = int(pred[i, j]) + 0.5
        return pred
    

if __name__ == '__main__':
    data_path = '../datasets/'
    data = np.load(data_path + 'ratings_train.npy')
    test_data = np.load(data_path + 'ratings_test.npy')
    
    print(np.unique(data))
    
    np.random.seed(42)
    
    t_1 = time.time()
    solver = Solve(k=3, mu = 0.02, alpha = 0.0005, beta = 0.0005, train_data=data, n_steps=50)
    pred = solver.train()
    t_2 = time.time()
    print(f'Elapsed time solver without mask: {t_2 - t_1}')
    rmse = solver.rmse(test_data)
    train_rmse = solver.rmse(data)
    print("\nGD Solver no mask")
    print(f"RMSE against TRAIN: {train_rmse}")
    print(f"RMSE against TEST: {rmse}")
    table = solver.predict()
    print(table)
    print(np.unique(table))
    
    '''
    t_1 = time.time()
    solver_2 = Solve(k=3, mu = 0.0002, alpha = 0.00005, beta = 0.000005, train_data=data, n_steps=50)
    pred = solver_2.train_masked()
    t_2 = time.time()
    print(f'\nElapsed time solver with mask: {t_2 - t_1}')
    rmse = solver_2.rmse(test_data)
    train_rmse = solver_2.rmse(data)
    print("\nSolver mask")
    print(f"RMSE against TRAIN: {train_rmse}")
    print(f"RMSE against TEST: {rmse}")
    '''
    
    t_1 = time.time()
    solver_als = Solve(k=1, mu=0.02, alpha=0.0005, beta=0.0005, train_data=data, n_steps=50)
    pred = solver_als.matrix_completion_als()
    t_2 = time.time()
    print(f'\nElapsed time ALS solver: {t_2 - t_1}')
    
    rmse = solver_als.rmse_als(test_data)
    train_rmse = solver_als.rmse_als(data)
    print("\nALS Solver")
    print(f"RMSE against TRAIN: {train_rmse}")
    print(f"RMSE against TEST: {rmse}")
    table_als = solver_als.predict_als()
    print(table_als)
    print(np.unique(table_als))
    
    rmse2 = solver_als.rmse_als2(test_data)
    train_rmse2 = solver_als.rmse_als2(data)
    print(f"RMSE 2 against TRAIN: {train_rmse2}")
    print(f"RMSE 2 against TEST: {rmse2}")
    table_als2 = solver_als.predict_als2()
    print(table_als2)

[0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  nan]
Elapsed time solver without mask: 12.19159746170044

GD Solver no mask
RMSE against TRAIN: 0.8877074735048488
RMSE against TEST: 1.0054679924874304
[[4.5 4.  4.  ... 4.  3.  3. ]
 [3.5 3.  2.5 ... 3.  2.  2. ]
 [1.5 1.5 1.5 ... 1.  1.  1. ]
 ...
 [3.5 3.5 3.5 ... 3.  2.5 2.5]
 [3.5 3.  2.5 ... 3.  2.  2. ]
 [4.5 4.  4.  ... 4.  3.  3. ]]
[1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Elapsed time ALS solver: 26.119940519332886

ALS Solver
RMSE against TRAIN: 0.7661019164173758
RMSE against TEST: 0.9178510072712412
[[4.5 4.  3.5 ... 4.5 4.  4.5]
 [4.  3.5 3.  ... 3.5 3.  3.5]
 [1.5 1.5 1.  ... 1.5 1.5 1.5]
 ...
 [3.5 3.  3.  ... 3.5 3.  3.5]
 [3.5 3.  3.  ... 3.5 3.  3.5]
 [4.  3.5 3.5 ... 4.  3.5 4. ]]
[1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]
RMSE 2 against TRAIN: 0.7887910229769244
RMSE 2 against TEST: 0.9419172209607792
[[5.  4.  4

In [16]:
import numpy as np

def initialize_with_svd(Y, rank):
    # Fill missing values with zeros
    Y_filled = np.nan_to_num(Y)

    # Perform SVD
    I, Sigma, Ut = np.linalg.svd(Y_filled, full_matrices=False)

    # Take top `rank` singular vectors/values
    I_init = I[:, :rank]
    Sigma_init = np.diag(Sigma[:rank])
    Ut_init = Ut[:rank, :]

    # Initialize I and U
    I_initialized = I_init @ np.sqrt(Sigma_init)
    U_initialized = (np.sqrt(Sigma_init) @ Ut_init).T

    return I_initialized, U_initialized

def matrix_completion_als(data, rank, max_iter, tol, lambda_reg):
    m, n = data.shape
    I = np.random.rand(m, rank)
    U = np.random.rand(n, rank)
    #I, U = initialize_with_svd(data, rank)
    error = 1e10
    
    Omega = (data > 0).astype(int)
    #R = np.nan_to_num(data, copy=True)
    
    for _ in range(max_iter):
        # Update U while fixing V
        for i in range(m):
            indices = np.where(Omega[i, :] == 1)[0]
            Vi = U[indices, :]
            Yi = data[i, indices]
            I[i, :] = np.linalg.solve(Vi.T @ Vi + lambda_reg * np.eye(rank), Vi.T @ Yi)
        
        # Update V while fixing U
        for j in range(n):
            indices = np.where(Omega[:, j] == 1)[0]
            Uj = I[indices, :]
            Yj = data[indices, j]
            U[j, :] = np.linalg.solve(Uj.T @ Uj + lambda_reg * np.eye(rank), Uj.T @ Yj)

        # Compute the matrix approximation and error
        Y_hat = I @ U.T
        diff = Omega * (data - Y_hat)
        new_error = np.linalg.norm(diff, 'fro')
        if abs(new_error - error) < tol:
            break
        error = new_error
    
    return np.clip(np.around((I@U.T)*2, 0)/2, 1, 5)


from scipy.sparse import find

def k_fold_split(data, k):
    """
    Split the observed entries of the sparse data matrix into k-folds.
    """
    i, j, v = find(data)  # Returns row, column, and value of non-zero entries
    observed_entries = list(zip(i, j))
    np.random.shuffle(observed_entries)
    
    split_size = len(observed_entries) // k
    folds = [observed_entries[i*split_size:(i+1)*split_size] for i in range(k)]
    
    return folds



from scipy.sparse import csr_matrix

def als_k_fold(data, k=5, max_iter=125, tol=1e-6, lambda_reg=0.34):
    m, n = data.shape
    k_folds = k_fold_split(data, k)
    
    # Assuming the same latent factor size 'k' for both user and item matrices
    latent_k = 10  # Adjust this value as needed
    predictions = np.zeros((k, m, n))
    
    for fold_idx, validation_fold in enumerate(k_folds):
        validation_rows, validation_cols = zip(*validation_fold)
        Omega = csr_matrix((np.ones(len(validation_fold)), (validation_rows, validation_cols)), shape=(m, n))
        
        I_2 = np.random.rand(m, latent_k)
        U_2 = np.random.rand(n, latent_k)
        
        for _ in range(max_iter):
            # Update I_2 while fixing U_2
            for i in range(m):
                indices = Omega[i].indices
                Vi = U_2[indices, :]
                Yi = data[i, indices].A1   # Convert to 1D numpy array
                I_2[i, :] = np.linalg.solve(Vi.T @ Vi + lambda_reg * np.eye(latent_k), Vi.T @ Yi)
                
            # Update U_2 while fixing I_2
            for j in range(n):
                indices = Omega[:, j].indices
                Uj = I_2[indices, :]
                Yj = data[indices, j].A1   # Convert to 1D numpy array
                U_2[j, :] = np.linalg.solve(Uj.T @ Uj + lambda_reg * np.eye(latent_k), Uj.T @ Yj)

        # Store the prediction for this fold
        predictions[fold_idx] = I_2 @ U_2.T
        
    # Average the predictions
    averaged_prediction = predictions.mean(axis=0)
    
    return averaged_prediction




data_path = '../datasets/'
data = np.load(data_path + 'ratings_train.npy')
test_data = np.load(data_path + 'ratings_test.npy')


# Create the Omega matrix: 1 where Y has an entry, 0 otherwise
#Omega = (data > 0).astype(int)

# Display the generated matrix
print("Original matrix (with missing values set to 0):")
print(data)


# Use the ALS method to complete the matrix
completed_matrix = matrix_completion_als(data, rank=1, max_iter=125, tol=1e-6, lambda_reg=0.34)

matrix_2 = als_k_fold(data, k=5, max_iter=125, tol=1e-6, lambda_reg=0.34)

print("\nCompleted matrix:")
print(completed_matrix)


def rmse(completed_matrix, test_matrix):
    masked = np.ma.array(test_matrix, mask=np.isnan(test_matrix))
    predictions = completed_matrix
    diff = np.ma.subtract(predictions, masked)
    squared = np.ma.power(diff, 2)
    return np.ma.sqrt(np.ma.mean(squared))

print(rmse(completed_matrix, data))
print(rmse(completed_matrix, test_data))

print(matrix_2)
print(rmse(matrix_2, data))
print(rmse(matrix_2, test_data))

Original matrix (with missing values set to 0):
[[ 4. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan  2. nan ... nan nan nan]
 [ 3. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


AttributeError: 'numpy.ndarray' object has no attribute 'A1'

In [ ]:
from sklearn.base import BaseEstimator
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

class ALSMatrixCompletion(BaseEstimator):
    def __init__(self, rank=1, max_iter=100, tol=1e-6, lambda_reg=0.1):
        self.rank = rank
        self.max_iter = max_iter
        self.tol = tol
        self.lambda_reg = lambda_reg
        self.filled_matrix = None

    def fit(self, X, y=None):
        self.filled_matrix = self.matrix_completion_als(X, self.rank, self.max_iter, self.tol, self.lambda_reg)
        return self

    def predict(self, X):
        return self.filled_matrix

    @staticmethod
    def matrix_completion_als(data, rank, max_iter, tol, lambda_reg):
        m, n = data.shape
        I = np.random.rand(m, rank)
        U = np.random.rand(n, rank)
        #I, U = initialize_with_svd(data, rank)
        error = 1e10
    
        Omega = (data > 0).astype(int)
        #R = np.nan_to_num(data, copy=True)
    
        for _ in range(max_iter):
            # Update U while fixing V
            for i in range(m):
                indices = np.where(Omega[i, :] == 1)[0]
                Vi = U[indices, :]
                Yi = data[i, indices]
                I[i, :] = np.linalg.solve(Vi.T @ Vi + lambda_reg * np.eye(rank), Vi.T @ Yi)
        
            # Update V while fixing U
            for j in range(n):
                indices = np.where(Omega[:, j] == 1)[0]
                Uj = I[indices, :]
                Yj = data[indices, j]
                U[j, :] = np.linalg.solve(Uj.T @ Uj + lambda_reg * np.eye(rank), Uj.T @ Yj)

            # Compute the matrix approximation and error
            Y_hat = I @ U.T
            diff = Omega * (data - Y_hat)
            new_error = np.linalg.norm(diff, 'fro')
            if abs(new_error - error) < tol:
                break
            error = new_error
    
        return np.clip(np.around((I@U.T)*2, 0)/2, 1, 5)


def rmse_b(true_matrix, predicted_matrix):
    min_rows = min(true_matrix.shape[0], predicted_matrix.shape[0])
    min_cols = min(true_matrix.shape[1], predicted_matrix.shape[1])

    true_submatrix = true_matrix[:min_rows, :min_cols]
    predicted_submatrix = predicted_matrix[:min_rows, :min_cols]

    non_zero_indices = true_submatrix.nonzero()  
    error = true_submatrix[non_zero_indices] - predicted_submatrix[non_zero_indices]
    return np.sqrt(np.mean(error**2))

def rmse_a(true_matrix, predicted_matrix):
    non_zero_indices = true_matrix.nonzero()  
    error = true_matrix[non_zero_indices] - predicted_matrix[non_zero_indices]
    return np.sqrt(np.mean(error**2))


def rmse_scorer(estimator, X):
    predicted = estimator.predict(X)
    return -rmse_a(X, predicted)  

def train_val_split(data, val_percentage=0.1):
    """
    Split the data into training and validation by masking a fraction of known entries.
    """
    nonzero_inds = data.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    num_samples = int(np.ceil(val_percentage * len(nonzero_pairs)))
    samples = np.random.choice(len(nonzero_pairs), num_samples, replace=False)
    val_inds = [nonzero_pairs[i] for i in samples]
    train_data = data.copy()
    for i, j in val_inds:
        train_data[i, j] = 0
    val_data = np.zeros(data.shape)
    for i, j in val_inds:
        val_data[i, j] = data[i, j]
    return train_data, val_data


param_grid = {
    'rank': [1],
    'max_iter': [50, 100],
    'tol': [1e-6],
    'lambda_reg': [0.01, 0.1]
}

grid_result = GridSearchCV(ALSMatrixCompletion(), param_grid, scoring=rmse_scorer, verbose=2)


train_data, val_data = train_val_split(data)

grid_result = GridSearchCV(ALSMatrixCompletion(), param_grid, scoring=rmse_scorer, verbose=2)

grid_result.fit(train_data)

# Evaluate using validation data
val_predictions = grid_result.predict(val_data)
print("Validation RMSE:", rmse_a(val_data, val_predictions))


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....lambda_reg=0.01, max_iter=50, rank=1, tol=1e-06; total time=   7.1s
[CV] END ....lambda_reg=0.01, max_iter=50, rank=1, tol=1e-06; total time=   7.1s
[CV] END ....lambda_reg=0.01, max_iter=50, rank=1, tol=1e-06; total time=  16.7s
[CV] END ....lambda_reg=0.01, max_iter=50, rank=1, tol=1e-06; total time=  11.4s
[CV] END ....lambda_reg=0.01, max_iter=50, rank=1, tol=1e-06; total time=  11.5s
[CV] END ...lambda_reg=0.01, max_iter=100, rank=1, tol=1e-06; total time=  22.5s
[CV] END ...lambda_reg=0.01, max_iter=100, rank=1, tol=1e-06; total time=  19.2s
[CV] END ...lambda_reg=0.01, max_iter=100, rank=1, tol=1e-06; total time=  13.3s
[CV] END ...lambda_reg=0.01, max_iter=100, rank=1, tol=1e-06; total time=  15.7s
[CV] END ...lambda_reg=0.01, max_iter=100, rank=1, tol=1e-06; total time=  13.7s
[CV] END .....lambda_reg=0.1, max_iter=50, rank=1, tol=1e-06; total time=   6.8s
[CV] END .....lambda_reg=0.1, max_iter=50, rank=1

In [ ]:
param_grid = {
    'rank': [1, 2, 3],
    'max_iter': [50, 75, 100],
    'tol': [1e-5, 1e-6],
    'lambda_reg': [0.01, 0.1]
}

grid_search = GridSearchCV()